In [2]:
import tensorflow as tf
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import scipy.stats as si

In [3]:
def bs(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
            
    delta = si.norm.cdf(d1)
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
                    
    return call, delta

In [4]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./mjd/mjd_b0.02105.ckpt.meta')
    saver.restore(sess, "./mjd/mjd_b0.02105.ckpt")
    print("Model restored.")

    # Cost graph
    X_train = np.ones((1000,3))
    X_train[:, 0] = np.linspace(80, 130, num=1000) # underlying asset price S
    X_train[:, 1] *= np.linspace(50, 120, num=1000) # strike price K
    X_train[:, 2] *= 90/250 # time to expiry T

    S_matrix, K_matrix = np.meshgrid(X_train[:,0], X_train[:,1])
    bs_call, bs_delta = bs(S_matrix, K_matrix, 90/250, 0, .15)

    # Plot Black-Scholes
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d', label='Black-Scholes')
    ax.plot_surface(S_matrix, K_matrix, bs_call)
    ax.set_xlabel('Underlying asset price')
    ax.set_ylabel('Strike price')
    ax.set_zlabel('Option price')

    # Plot MJD
    #mjd_call, mjd_delta = mjd(X_train[:,0], 100, 90/250, 0, 0.1114, 64, -1.145e-3, 1.505e-2)
    mjd_call, mjd_delta = mjd(S_matrix, K_matrix, 90/250, 0, 0.1114, 64, -1.145e-3, 1.505e-2 + 0.006)
    ax.plot_surface(S_matrix, K_matrix, mjd_call)

    # Plot neural network
    X_train = np.stack((S_matrix.flatten(), K_matrix.flatten(), np.full((1000000), 90/250)), axis=-1)
    out_values = sess.run('ann/mul:0', feed_dict={'inputs_processing/X_input:0': X_train})
    S_matrix_nn = np.reshape(X_train[:,0], (1000,1000))
    K_matrix_nn = np.reshape(X_train[:,1], (1000,1000))
    call_nn = np.reshape(out_values, (1000,1000))
    ax.plot_surface(S_matrix_nn, K_matrix_nn, call_nn)
    plt.show()


AttributeError: module 'tensorflow' has no attribute 'Session'